In [1]:
from random import choice

In [2]:
class Die():
    FACES = ['🎲', '⚀', '⚁', '⚂', '⚃', '⚄', '⚅']
    
    def __init__(self):
        self.value = 0
        self.fixed = False
    
    def __str__(self):
        return self.FACES[self.value]
        
    def roll(self):
        if not (self.fixed):
            self.value = choice(range(1,7))
        return self.value
    
    def get_value(self):
        return self.value
    
    def toggle(self):
        self.fixed = not self.fixed
        return self.fixed
    
    def get_fixed(self):
        return self.fixed
    
    def set_fixed(self, flag):
        self.fixed = flag
        return

In [3]:
class Dice(Die):
    def __init__(self):
        self.dice = []
        for i in range(5):
            self.dice.append(Die())
        return
    
    def __str__(self):
        string = ''
        for idx in range(len(self.dice)):
            string += (str(self.dice[idx]) + ' ')
        return string.strip()
    
    def roll(self):
        for die in self.dice:
            die.roll()
        return
    
    def set_fixed(self, flags):
        for idx, die in enumerate(self.dice):
            if idx in flags:
                die.set_fixed(True)
            else:
                die.set_fixed(False)
        return
    
    def get_dice_value(self):
        value = []
        for die in self.dice:
            value.append(die.get_value())
        return value
    
    def reset_fixed(self):
        for die in self.dice:
            die.set_fixed(False)

In [14]:
class Scores():
    CHOICES = ['Fix dice', 'Ones', 'Twos', 'Threes', 'Fours', 
               'Fives', 'Sixes', 'Full House', 'Four same', 
               'Little Straight', 'Big Straight', 'Choice', 
               'Yacht']
    
    def __init__(self):
        self.categories = [-1] * 13
        return
    
    def score_dice(self, category, dice):
        score = 0
        if 0 < category <= 6: 
            # First six categories
            for die in dice:
                if die == category:
                    score += category
                    
        elif category == 7 or category == 8:
            # Full house and four of the same
            first_count = {'value':dice[0], 'count':0}
            second_count = {'value':0, 'count':0}
            for die in dice:
                if die == first_count['value']:
                    first_count['count'] += 1
                elif second_count['value'] == 0:
                    second_count['value'] = die
                    second_count['count'] += 1
                elif die == second_count['value']:
                    second_count['count'] += 1
                else:
                    return 0
            if category == 7: 
                # full house
                if ((first_count['count']==2 or first_count['count']==3)
                   and (first_count['count'] + second_count['count'] == 5)):
                    score = sum(dice)
            else: 
                # four of the same
                if first_count['count'] >= 4:
                    score = first_count['value'] * first_count['count']
                elif second_count['count'] >= 4:
                    score = second_count['value'] * second_count['count']
                    
        elif category == 9 or category == 10: 
            # Little straight & big straight 
            dice.sort()
            straight_flag = True
            for idx in range(4):
                if dice[idx] != dice[idx+1]-1:
                    straight_flag = False
            if straight_flag:
                if (((category == 9) and (dice[0] == 1))
                    or ((category == 10) and (dice[0] == 2))):
                    score = 30
                    
        elif category == 11:
            # Choice
            score = sum(dice)
            
        elif category == 12:
            # Yacht
            if dice[0] == 0:
                return 0
            if dice.count(dice[0]) == len(dice):
                score = 50
        return score
    
    def get_score(self, category):
        return self.categories[category]
    
    def set_score(self, category, score):
        if self.categories[category] == -1:
            self.categories[category] = score
            return True
        return False
    
    def is_game_over(self):
        return all(cat >= 0 for cat in self.categories[1:])

In [15]:
class Yacht(Dice, Scores):
    def __init__(self):
        self.dice = Dice()
        self.scores = Scores()
        self.rolled_turns = 0
        self.score = 0 
        
    def reset_game(self):
        self.__init__()
        return

    def print_dice(self):
        print('')
        print(self.dice)
        return
        
    def roll_dice(self):
        if self.rolled_turns >= 3:
            return
        else:
            self.rolled_turns += 1
            
        self.dice.roll()
        self.print_dice()
        return
    
    def fix_dice(self):
        print('0 1 2 3 4')
        print(self.dice)
        to_fix = input('Please enter dice index/indices to fix separated by space:').split()
        self.dice.set_fixed([int(i) for i in to_fix])
        return
    
    def display_choices(self):
        for idx, choice in enumerate(self.CHOICES):
            if (idx == 0):
                if 0 < self.rolled_turns < 3:
                    print('Enter to roll dice.')
                    print(str(idx) + '. ' + choice)
            else:
                if self.scores.get_score(idx) >= 0:
                    print(str(idx) + '. ' + choice + ' scored: ' 
                          + str(self.scores.get_score(idx)))
                else:
                    print(str(idx) + '. ' + choice + ' will score: ' 
                          + str(self.scores.score_dice(idx, self.dice.get_dice_value())))
        print('Current score: ' + str(self.score))
        return
    
    def play_game(self):
        loop = True
        while loop:
            self.display_choices()
            if self.rolled_turns == 0:
                input('Press Enter to roll dice. ')
                self.roll_dice()
                continue
            else:
                user_entry = input('Select: ')
            if len(user_entry) == 0:
                self.roll_dice()
            else:
                try:
                    selection = int(user_entry)
                except ValueError:
                    continue
                if selection == 0:
                    self.fix_dice()
                elif selection < len(self.CHOICES):
                    if self.scores.categories[selection] < 0:
                        score = self.scores.score_dice(selection, self.dice.get_dice_value())
                        self.score += score
                        self.scores.set_score(selection, score) 
                        self.rolled_turns = 0 # reset rolled_turns
                        self.dice.reset_fixed()
                        if self.scores.is_game_over():
                            loop = False
        print('\nGame Over! You Scored ' + str(self.score) + ' !')
        return

In [16]:
game = Yacht()

In [18]:
game.play_game()

1. Ones scored: 2
2. Twos scored: 2
3. Threes scored: 3
4. Fours will score: 4
5. Fives will score: 0
6. Sixes will score: 6
7. Full House scored: 27
8. Four same scored: 24
9. Little Straight will score: 0
10. Big Straight will score: 0
11. Choice will score: 16
12. Yacht will score: 0
Current score: 58
Press Enter to roll dice. 

⚅ ⚃ ⚅ ⚄ ⚁
Enter to roll dice.
0. Fix dice
1. Ones scored: 2
2. Twos scored: 2
3. Threes scored: 3
4. Fours will score: 4
5. Fives will score: 5
6. Sixes will score: 12
7. Full House scored: 27
8. Four same scored: 24
9. Little Straight will score: 0
10. Big Straight will score: 0
11. Choice will score: 23
12. Yacht will score: 0
Current score: 58
Select: 0
0 1 2 3 4
⚅ ⚃ ⚅ ⚄ ⚁
Please enter dice index/indices to fix separated by space:1 2 3 4
Enter to roll dice.
0. Fix dice
1. Ones scored: 2
2. Twos scored: 2
3. Threes scored: 3
4. Fours will score: 4
5. Fives will score: 5
6. Sixes will score: 12
7. Full House scored: 27
8. Four same scored: 24
9. Little Stra

Please enter dice index/indices to fix separated by space:0 1 2 4
Enter to roll dice.
0. Fix dice
1. Ones scored: 2
2. Twos scored: 2
3. Threes scored: 3
4. Fours scored: 4
5. Fives scored: 20
6. Sixes will score: 12
7. Full House scored: 27
8. Four same scored: 24
9. Little Straight scored: 30
10. Big Straight scored: 30
11. Choice will score: 18
12. Yacht will score: 0
Current score: 142
Select: 

⚀ ⚅ ⚅ ⚅ ⚀
1. Ones scored: 2
2. Twos scored: 2
3. Threes scored: 3
4. Fours scored: 4
5. Fives scored: 20
6. Sixes will score: 18
7. Full House scored: 27
8. Four same scored: 24
9. Little Straight scored: 30
10. Big Straight scored: 30
11. Choice will score: 20
12. Yacht will score: 0
Current score: 142
Select: 6
1. Ones scored: 2
2. Twos scored: 2
3. Threes scored: 3
4. Fours scored: 4
5. Fives scored: 20
6. Sixes scored: 18
7. Full House scored: 27
8. Four same scored: 24
9. Little Straight scored: 30
10. Big Straight scored: 30
11. Choice will score: 20
12. Yacht will score: 0
Current sc